In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/train.csv')
df.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [3]:
df.isnull().sum()

bant_submit                    0
customer_country             982
business_unit                  0
com_reg_ver_win_rate       44731
customer_idx                   0
customer_type              43961
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver           55855
it_strategic_ver           58178
idit_strategic_ver         54734
customer_job               18733
lead_desc_length               0
inquiry_type                 941
product_category           19374
product_subcategory        50064
product_modelname          50070
customer_country.1           982
customer_position              0
response_corporate             0
expected_timeline          30863
ver_cus                        0
ver_pro                        0
ver_win_rate_x             40882
ver_win_ratio_per_bu       43995
business_area              40882
business_subarea           53773
lead_owner                     0
is_converted                   0
dtype: int64

### 컬럼 제거

In [80]:
col = ['customer_country', 'customer_idx', 'customer_job', 'product_category', 'product_subcategory', 'product_modelname', 'customer_country.1', 'expected_timeline', 'business_area', 'business_subarea', 'lead_owner']
df.drop(columns=col, inplace = True)

In [81]:
df.isnull().sum()

bant_submit                    0
business_unit                  0
com_reg_ver_win_rate       44731
customer_type              43961
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver           55855
it_strategic_ver           58178
idit_strategic_ver         54734
lead_desc_length               0
inquiry_type                 941
customer_position              0
response_corporate             0
ver_cus                        0
ver_pro                        0
ver_win_rate_x             40882
ver_win_ratio_per_bu       43995
is_converted                   0
dtype: int64

In [82]:
df['bant_submit'].unique()

array([1.  , 0.75, 0.5 , 0.25, 0.  ])

In [36]:
df['business_unit'].unique()

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [84]:
df['com_reg_ver_win_rate'].fillna(0, inplace = True)

In [87]:
df['customer_type'] = df['customer_type'].str.lower()
df['customer_type'] = df['customer_type'].replace(['etc.', 'other', 'others'], 'etc')
df['customer_type'] = df['customer_type'].replace(['end-customer', 'end customer', 'end-user'], 'end_user')
df['customer_type'] = df['customer_type'].replace(['specifier/ influencer', 'specifier / influencer'], 'influencer')
df['customer_type'] = df['customer_type'].replace(['homeowner', 'home owner'], 'home_owner')
df['customer_type'] = df['customer_type'].replace(['software/solution provider', 'software / solution provider'], 'solution_provider')

# 값의 빈도 계산
value_counts = df['customer_type'].value_counts()

# 10개 미만인 값들을 식별
values_to_replace = value_counts[value_counts < 100].index

# 해당 값들을 'etc'로 대체
df['customer_type'] = df['customer_type'].apply(lambda x: 'etc' if x in values_to_replace else x)

In [109]:
df['customer_type'].fillna('etc', inplace = True)

In [111]:
df['customer_type'].value_counts()

customer_type
etc                     44217
end_user                10651
influencer               2568
channel partner          1368
service partner           349
solution eco-partner      146
Name: count, dtype: int64

In [90]:
df['enterprise'].unique()

array(['Enterprise', 'SMB'], dtype=object)

In [91]:
df['historical_existing_cnt'].fillna(0, inplace = True)

In [92]:
df[['it_strategic_ver', 'id_strategic_ver', 'idit_strategic_ver']] = df[['it_strategic_ver', 'id_strategic_ver', 'idit_strategic_ver']].fillna(0)

In [93]:
df['inquiry_type'] = df['inquiry_type'].str.lower()
df['inquiry_type'] = df['inquiry_type'].replace(['etc.', 'other', 'other_', 'others'], 'etc')
df['inquiry_type'] = df['inquiry_type'].replace(['quotation or purchase consultation', 'request for quotation or purchase', 'quotation_or_purchase_consultation', 'purchase or quotation', 'quotation_'], 'quotation_or_purchase_consultation')
df['inquiry_type'] = df['inquiry_type'].replace(['usage or technical consultation', 'technical consultation', 'request for technical consulting', 'usage_or_technical_consultation', 'technical_consultation', 'technical'], 'usage or technical consultation')

In [95]:
# 값의 빈도 계산
value_counts = df['inquiry_type'].value_counts()

# 100개 미만인 값들을 식별
values_to_replace = value_counts[value_counts < 1000].index

# 해당 값들을 'etc'로 대체
df['inquiry_type'] = df['inquiry_type'].apply(lambda x: 'etc' if x in values_to_replace else x)

In [104]:
df['inquiry_type'].fillna('etc', inplace = True)

In [107]:
df['inquiry_type'].value_counts()

inquiry_type
quotation_or_purchase_consultation    42379
sales inquiry                          9981
etc                                    4034
usage or technical consultation        1668
product information                    1237
Name: count, dtype: int64

In [98]:
# 값의 빈도 계산
df['customer_position'] = df['customer_position'].replace(['other', 'others'], 'etc')
value_counts = df['customer_position'].value_counts()
values_to_replace = value_counts[value_counts < 1000].index
df['customer_position'] = df['customer_position'].apply(lambda x: 'etc' if x in values_to_replace else x)

In [100]:
df['customer_position'].value_counts()

customer_position
none                 19680
etc                  11226
manager               8217
ceo/founder           7984
director              4843
associate/analyst     2597
partner               2536
entry level           2216
Name: count, dtype: int64

In [77]:
df['response_corporate'].unique()

array(['LGEPH', 'LGEIL', 'LGEAF', 'LGESJ', 'LGESL', 'LGESP', 'LGEGF',
       'LGESA', 'LGEUS', 'LGECB', 'LGEMS', 'LGEEG', 'LGEEF', 'LGEAP',
       'LGEIN', 'LGEUK', 'LGEKR', 'LGEPS', 'LGECI', 'LGECL', 'LGETK',
       'LGELF', 'LGEPT', 'LGEPR', 'LGEDG', 'LGERO', 'LGEMK', 'LGEPL',
       'LGECZ', 'LGEES', 'LGEAR', 'LGEHK', 'LGEML', 'LGEJP', 'LGEHS',
       'LGEAS', 'LGEYK', 'LGEIS', 'LGEBN', 'LGEFS', 'LGESW', 'LGEMC',
       'LGEAG', 'LGEEB', 'LGETH', 'LGEVH', 'LGECH', 'LGELA', 'LGETT',
       'LGERA', 'LGEUR', 'LGEIR', 'LGEBT'], dtype=object)

In [101]:
df['ver_win_rate_x'].fillna(0, inplace = True)
df['ver_win_ratio_per_bu'].fillna(0, inplace = True)

In [13]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [112]:
df.isnull().sum()

bant_submit                0
business_unit              0
com_reg_ver_win_rate       0
customer_type              0
enterprise                 0
historical_existing_cnt    0
id_strategic_ver           0
it_strategic_ver           0
idit_strategic_ver         0
lead_desc_length           0
inquiry_type               0
customer_position          0
response_corporate         0
ver_cus                    0
ver_pro                    0
ver_win_rate_x             0
ver_win_ratio_per_bu       0
is_converted               0
dtype: int64

In [113]:
df.head()

,bant_submit,business_unit,com_reg_ver_win_rate,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,lead_desc_length,inquiry_type,customer_position,response_corporate,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,is_converted
0,1.0,AS,0.066667,end_user,Enterprise,0.0,0.0,0.0,0.0,62,quotation_or_purchase_consultation,entry level,LGEPH,1,0,0.003079,0.026846,True
1,1.0,AS,0.066667,end_user,Enterprise,12.0,0.0,0.0,0.0,96,quotation_or_purchase_consultation,ceo/founder,LGEPH,1,0,0.003079,0.026846,True
2,1.0,AS,0.088889,end_user,Enterprise,144.0,0.0,0.0,0.0,56,product information,partner,LGEIL,1,0,0.003079,0.026846,True
3,1.0,AS,0.088889,end_user,Enterprise,0.0,0.0,0.0,0.0,44,quotation_or_purchase_consultation,ceo/founder,LGEIL,1,0,0.003079,0.026846,True
4,1.0,AS,0.088889,influencer,Enterprise,0.0,0.0,0.0,0.0,97,quotation_or_purchase_consultation,partner,LGEIL,0,0,0.003079,0.026846,True


In [ ]:
# 그래도 쓸 컬럼
['bant_submit',
 'com_reg_ver_win_rate',
 'historical_existing_cnt',
 'it_strategic_ver',
 'id_strategic_ver',
 'idit_strategic_ver',
 'ver_cus',
 'ver_pro',
 'ver_win_rate_x',
 'ver_win_ratio_per_bu'
]

In [115]:
# 레이블 인코딩할 칼럼들
label_columns = [
    'business_unit',
    'customer_type',
    'enterprise',
    'response_corporate',
    'inquiry_type',
    'customer_position',
    'response_corporate',
]

In [116]:
for col in label_columns:
    df[col] = label_encoding(df[col])

In [118]:
df.to_csv('data/pre_train.csv')

In [120]:
data = pd.read_csv('data/pre_train.csv', index_col=0)
data

,bant_submit,business_unit,com_reg_ver_win_rate,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,lead_desc_length,inquiry_type,customer_position,response_corporate,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,is_converted
0,1.00,0,0.066667,1,0,0.0,0.0,0.0,0.0,62,2,3,27,1,0,0.003079,0.026846,True
1,1.00,0,0.066667,1,0,12.0,0.0,0.0,0.0,96,2,1,27,1,0,0.003079,0.026846,True
2,1.00,0,0.088889,1,0,144.0,0.0,0.0,0.0,56,1,7,14,1,0,0.003079,0.026846,True
3,1.00,0,0.088889,1,0,0.0,0.0,0.0,0.0,44,2,1,14,1,0,0.003079,0.026846,True
4,1.00,0,0.088889,3,0,0.0,0.0,0.0,0.0,97,2,7,14,0,0,0.003079,0.026846,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,0,0.000000,1,1,0.0,0.0,0.0,0.0,200,1,3,28,0,0,0.000026,0.028777,False
59295,0.75,0,0.040000,3,0,0.0,0.0,0.0,0.0,70,0,4,50,0,0,0.000026,0.028777,False
59296,0.75,0,0.040000,3,0,0.0,0.0,0.0,0.0,34,0,3,29,0,0,0.000026,0.028777,False
59297,1.00,0,0.040000,2,0,0.0,0.0,0.0,0.0,377,2,4,29,0,0,0.000026,0.028777,False
